In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#you may need to install bs4 on your system, uncomment below line to do so
#pip install beautifulsoup4
from bs4 import BeautifulSoup
import geocoder
print('Libraries imported.')

Libraries imported.


In [3]:
df=pd.read_csv('Toronto Data Final.csv')
df.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [7]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [10]:
print(df.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64


#### Create a map of Toronto and look at neighborhoods

In [8]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [9]:
map_data = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_data)  
    
map_data

#### I am going to focus on the Borough of "Etobicoke" because the name is awesome

In [11]:
e_data = df[df['Borough'] == 'Etobicoke'].reset_index(drop=True)
e_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
3,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
4,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999


#### Create map of Etobicoke Borough

In [15]:
address = 'Etobicoke, Toronto'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.671459150000004, -79.55249206611668.


In [17]:
e_map_data = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(e_data['Latitude'], e_data['Longitude'], e_data['Borough'], e_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(e_map_data)  
    
e_map_data

#### Foursquare API info

In [18]:
CLIENT_ID = 'DV55GDUOOBL2ENGM3T0M53X3BEBEHFQOD002KFYXZTBBBD5Q' # your Foursquare ID
CLIENT_SECRET = 'AIKIXG0HUI2GXD1IIVSESHCAXUKVEFPOHKEXCA3MSM102NBL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
CODE = 'IABQW5UKAU2YROIX5IN2YBKQQ1UORYJ4QB53ITH4RKS3ABH1#_=_' # because we needed this earlier
ACCESS_TOKEN='20PXMW2PWFFPCNM0UCDU1M1O250XIPNEHNVIPI1BS0PSTA4P' #because we randomly needed this before


#### Select random neighborhood (4) and get nearby venues

In [23]:
neigh_latitude = e_data.loc[4, 'Latitude'] 
neigh_longitude = e_data.loc[4, 'Longitude'] # neighborhood longitude value

neigh_name = e_data.loc[4, 'PostalCode'] # postal code

print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_latitude, 
                                                               neigh_longitude))

Latitude and longitude values of M8Z are 43.6288408, -79.52099940000002.


#### Now we will get 50 venues in 500 meters

In [30]:
LIMIT = 50 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_latitude, 
    neigh_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=DV55GDUOOBL2ENGM3T0M53X3BEBEHFQOD002KFYXZTBBBD5Q&client_secret=AIKIXG0HUI2GXD1IIVSESHCAXUKVEFPOHKEXCA3MSM102NBL&v=20180605&ll=43.6288408,-79.52099940000002&radius=500&limit=50'

In [31]:
results = requests.get(url).json()

In [32]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Put results in a DataFrame and make it pretty

In [33]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,South St. Burger,Burger Joint,43.631314,-79.518408
1,Wingporium,Wings Joint,43.630275,-79.518169
2,Dollarama,Discount Store,43.629883,-79.518627
3,Healthy Planet,Supplement Shop,43.630214,-79.518495
4,Artisano Bakery Café,Bakery,43.631006,-79.518172


#### Considering 'Dollarama' is on the list we will wonder how many venues are in this area

In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

14 venues were returned by Foursquare.


In [35]:
print(e_data.loc[4, 'Neighborhood'])

Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West


#### Based on the above information, the above neighborhoods are not the happening places to be

#### Now see what we can learn about the rest of the neighborhoods in the borough

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Here are the groups (by postal code) of the neighborhoods

In [45]:
E_venues= getNearbyVenues(names=e_data['Neighborhood'],
                                   latitudes=e_data['Latitude'],
                                   longitudes=e_data['Longitude']
                                  )


New Toronto, Mimico South, Humber Bay Shores
Alderwood, Long Branch
The Kingsway, Montgomery Road, Old Mill North
Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East
Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West
Islington Avenue
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Westmount
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Northwest


#### Now get an idea of how many venues are in the area

In [46]:
print(E_venues.shape)
E_venues.head()

(79, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
1,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,New Toronto Fish & Chips,43.601849,-79.503281,Restaurant
2,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Domino's Pizza,43.601583,-79.500905,Pizza Place
3,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Delicia Bakery & Pastry,43.601403,-79.503012,Bakery
4,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Lucky Dice Restaurant,43.601392,-79.503056,Café


#### See what groups have more venues

In [47]:
E_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alderwood, Long Branch",11,11,11,11,11,11
"Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood",7,7,7,7,7,7
"Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens",4,4,4,4,4,4
"Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West",14,14,14,14,14,14
"New Toronto, Mimico South, Humber Bay Shores",13,13,13,13,13,13
Northwest,3,3,3,3,3,3
"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",3,3,3,3,3,3
"South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens",12,12,12,12,12,12
"The Kingsway, Montgomery Road, Old Mill North",4,4,4,4,4,4


#### Wide range on how many venues each area has

#### See how many different types of venues exist

In [48]:
print('There are {} uniques categories.'.format(len(E_venues['Venue Category'].unique())))

There are 48 uniques categories.


#### Now to see the type venues in the neighborhoods

In [51]:
# one hot encoding
E_onehot = pd.get_dummies(E_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
E_onehot['Neighborhood'] =E_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [E_onehot.columns[-1]] + list(E_onehot.columns[:-1])
E_onehot = E_onehot[fixed_columns]
E_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bar,Baseball Field,Beer Store,Brewery,Burger Joint,Burrito Place,Bus Line,Café,Chinese Restaurant,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Discount Store,Drugstore,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Grocery Store,Gym,Hardware Store,Intersection,Japanese Restaurant,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Seafood Restaurant,Skating Rink,Smoke Shop,Social Club,Supplement Shop,Tanning Salon,Wings Joint
0,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"New Toronto, Mimico South, Humber Bay Shores",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
E_group = E_onehot.groupby('Neighborhood').mean().reset_index()
E_group

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bar,Baseball Field,Beer Store,Brewery,Burger Joint,Burrito Place,Bus Line,Café,Chinese Restaurant,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Discount Store,Drugstore,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Grocery Store,Gym,Hardware Store,Intersection,Japanese Restaurant,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Seafood Restaurant,Skating Rink,Smoke Shop,Social Club,Supplement Shop,Tanning Salon,Wings Joint
0,"Alderwood, Long Branch",0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.090909,0.181818,0.090909,0.090909,0.000000,0.000000,0.00,0.090909,0.000000,0.090909,0.00,0.000000,0.000000,0.000000,0.000000
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.0,0.000000,0.000000,0.00,0.142857,0.000000,0.142857,0.000000,0.142857,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.00,0.00,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,"Kingsview Village, St. Phillips, Martin Grove ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
3,"Mimico NW, The Queensway West, South of Bloor,...",0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.0,0.071429,0.071429,0.00,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.071429,0.000000,0.071429,0.000000,0.000000,0.071429,0.071429,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.071429,0.000000,0.000000,0.00,0.071429,0.071429,0.071429,0.071429
4,"New Toronto, Mimico South, Humber Bay Shores",0.076923,0.000000,0.076923,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.076923,0.076923,0.000000,0.076923,0.000000,0.000000,0.000000,0.076923,0.076923,0.000000,0.00,0.00,0.076923,0.076923,0.000000,0.000000,0.000000,0.076923,0.00,0.000000,0.076923,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,Northwest,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
6,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
7,"South Steeles, Silverstone, Humbergate, Jamest...",0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.083333,0.000000,0.000000,0.0000

#### What are the top 5 types of venue for each neighborhood

In [53]:
num_top_venues = 5

for hood in E_group['Neighborhood']:
    print("----"+hood+"----")
    temp = E_group[E_group['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.18
1             Gym  0.09
2  Sandwich Place  0.09
3        Pharmacy  0.09
4    Dance Studio  0.09


----Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood----
            venue  freq
0    Liquor Store  0.14
1  Cosmetics Shop  0.14
2      Beer Store  0.14
3     Pizza Place  0.14
4            Café  0.14


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                 venue  freq
0    Mobile Phone Shop  0.25
1       Sandwich Place  0.25
2             Bus Line  0.25
3          Pizza Place  0.25
4  American Restaurant  0.00


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West----
               venue  freq
0                Gym  0.07
1  Convenience Store  0.07
2      Tanning Salon  0.07
3    Supplement Shop  0.07
4        Social Club  0.07


----New Toronto, Mimico South, Humber Bay Shores----
                  venue  freq
0

#### Learned that the majority of this Borough is not a "happening" place

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Since the area seems so sparse we will keep the venues as top 5

In [100]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = E_group['Neighborhood']

for ind in np.arange(E_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(E_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Dance Studio,Skating Rink,Gym
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Coffee Shop,Beer Store,Liquor Store,Cosmetics Shop,Convenience Store
2,"Kingsview Village, St. Phillips, Martin Grove ...",Sandwich Place,Mobile Phone Shop,Bus Line,Pizza Place,Wings Joint
3,"Mimico NW, The Queensway West, South of Bloor,...",Wings Joint,Hardware Store,Bakery,Burger Joint,Burrito Place
4,"New Toronto, Mimico South, Humber Bay Shores",American Restaurant,Pizza Place,Fast Food Restaurant,Flower Shop,Fried Chicken Joint


#### Yup, the Etobicoke Borough seems more blue collar than upscale

 #### Now we will cluster, since it is just the borough we will do k=4

In [108]:
# set number of clusters
kclusters = 4

E_grouped_clustering = E_group.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(E_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 0, 3, 2, 2, 1], dtype=int32)

In [101]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [109]:
E_merged = e_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
E_merged = E_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
E_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,2.0,American Restaurant,Pizza Place,Fast Food Restaurant,Flower Shop,Fried Chicken Joint
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,2.0,Pizza Place,Athletics & Sports,Dance Studio,Skating Rink,Gym
2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,2.0,Park,Smoke Shop,River,Pool,Wings Joint
3,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,3.0,Baseball Field,Deli / Bodega,Construction & Landscaping,Wings Joint,Coffee Shop
4,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,2.0,Wings Joint,Hardware Store,Bakery,Burger Joint,Burrito Place


#### Had to convert my cluster labels to int, still don't know why they were floats

In [128]:
E_merged['Cluster Labels'] =E_merged['Cluster Labels'].fillna(0).astype(int)

#### Here's the Map

In [129]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(E_merged['Latitude'], E_merged['Longitude'], E_merged['Neighborhood'], E_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [132]:
E_merged.loc[E_merged['Cluster Labels'] == 0, E_merged.columns[[1] + list(range(5, E_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Etobicoke,0,NaN,NaN,NaN,NaN,NaN
11,Etobicoke,0,Drugstore,Bar,Rental Car Location,Wings Joint,Chinese Restaurant


#### Cluster 2

In [133]:
E_merged.loc[E_merged['Cluster Labels'] == 1, E_merged.columns[[1] + list(range(5, E_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,Etobicoke,1,Brewery,Wings Joint,Coffee Shop,Flower Shop,Fast Food Restaurant


#### Cluster 3

In [134]:
E_merged.loc[E_merged['Cluster Labels'] == 2, E_merged.columns[[1] + list(range(5, E_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Etobicoke,2,American Restaurant,Pizza Place,Fast Food Restaurant,Flower Shop,Fried Chicken Joint
1,Etobicoke,2,Pizza Place,Athletics & Sports,Dance Studio,Skating Rink,Gym
2,Etobicoke,2,Park,Smoke Shop,River,Pool,Wings Joint
4,Etobicoke,2,Wings Joint,Hardware Store,Bakery,Burger Joint,Burrito Place
7,Etobicoke,2,Coffee Shop,Beer Store,Liquor Store,Cosmetics Shop,Convenience Store
8,Etobicoke,2,Pizza Place,Chinese Restaurant,Intersection,Sandwich Place,Middle Eastern Restaurant
9,Etobicoke,2,Sandwich Place,Mobile Phone Shop,Bus Line,Pizza Place,Wings Joint
10,Etobicoke,2,Grocery Store,Pizza Place,Pharmacy,Fast Food Restaurant,Discount Store


#### Cluster 4

In [136]:
E_merged.loc[E_merged['Cluster Labels'] == 3, E_merged.columns[[1] + list(range(5, E_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Etobicoke,3,Baseball Field,Deli / Bodega,Construction & Landscaping,Wings Joint,Coffee Shop


#### Probably could have gone with fewer clusters since it is a small area with few venues.  It appears you have a restaurant cluster and then you have other neighborhoods with other attractions, but nothing that is common amongst the other non-restaurant neighborhoods